In [1]:
%config Completer.use_jedi = False

In [3]:
import xpypact as xp

In [5]:
xp.__version__

'0.1.12a0'

In [11]:
from pathlib import Path

In [19]:
root_dir = Path("~", "dev", "xpypact").expanduser()

In [20]:
json_path = root_dir / "tests/data/Ag-1.json"
assert json_path.exists()

In [23]:
from xpypact.Inventory import Inventory, from_json
import xpypact.data_arrays as da

In [24]:
inventory = from_json(json_path)

In [25]:
ds = da.create_dataset(inventory)

In [26]:
ds

<xarray.Dataset>
Dimensions:                 (time_step_number: 2, nuclide: 48, timestamp: 1)
Coordinates:
  * time_step_number        (time_step_number) int64 1 2
  * nuclide                 (nuclide) object MultiIndex
  * element                 (nuclide) object 'Ag' 'Ag' 'Ag' ... 'Ru' 'Ru' 'Ru'
  * mass_number             (nuclide) int64 105 105 106 106 ... 106 102 103 104
  * state                   (nuclide) object '' 'm' '' 'm' '' ... 'm' '' '' ''
    elapsed_time            (time_step_number) float64 0.0 6.312e+07
    zai                     (nuclide) int64 471050 471051 ... 441030 441040
  * timestamp               (timestamp) datetime64[ns] 2020-07-12T23:01:19
Data variables: (12/31)
    irradiation_time        (time_step_number) float64 0.0 6.312e+07
    cooling_time            (time_step_number) float64 0.0 0.0
    duration                (time_step_number) float64 0.0 6.312e+07
    flux                    (time_step_number) float64 0.0 2.445e+10
    total_atoms             (time_step_number) float64 5.578e+21 5.578e+21
    total_activity          (time_step_number) float64 0.0 6.123e+09
    ...                      ...
    nuclide_alpha_heat      (time_step_number, nuclide) float64 nan nan ... 0.0
    nuclide_beta_heat       (time_step_number, nuclide) float64 nan nan ... 0.0
    nuclide_gamma_heat      (time_step_number, nuclide) float64 nan nan ... 0.0
    nuclide_dose            (time_step_number, nuclide) float64 nan nan ... 0.0
    nuclide_ingestion       (time_step_number, nuclide) float64 nan nan ... 0.0
    nuclide_inhalation      (time_step_number, nuclide) float64 nan nan ... 0.0
Attributes:
    run_name:            * Material Ag, fluxes 1
    flux_name:           55.F9.10 11-L2-02W HFS_GLRY_08_U
    dose_rate_type:      Point source
    dose_rate_distance:  1.0

In [28]:
import duckdb as db

In [46]:
db_path = root_dir / "wrk/try-duckdb.duckdb"
db_path.parent.mkdir(parents=True, exist_ok=True)

In [48]:
con = db.connect(str(db_path))

In [153]:
# con.execute("drop table rundata")
# con.execute("drop table if exists timestep_gamma")
con.execute("drop table if exists timestep_nuclide")
# con.execute("drop table if exists timestep")
con.execute("drop table nuclide")


In [154]:
def create_tables(con):
    sql = """
    
    CREATE TABLE IF NOT EXISTS rundata (
        timestamp varchar NOT NULL,
        run_name  varchar NOT NULL,
        flux_name varchar NOT NULL,
        dose_rate_type varchar NOT NULL,
        dose_rate_distance real NOT NULL
    );

    CREATE TABLE IF NOT EXISTS timestep(
        id integer PRIMARY KEY,
        elapsed_time real not null,
        irradiation_time real not null,
        cooling_time real not null,
        duration real not null,
        flux real not null,
        total_atoms real not null,
        total_activity real not null,
        total_alpha_activity real not null,
        total_beta_activity real not null,
        total_gamma_activity real not null,
        total_mass real not null,
        total_heat real not null,
        total_alpha_heat real not null,
        total_beta_heat real not null,
        total_gamma_heat real not null,
        total_ingest1ion_dose real not null,
        total_inhalation_dose real not null,
        total_dose_rate real not null
    );
    
    CREATE TABLE IF NOT EXISTS nuclide(
        element varchar(2) not null,
        mass_number integer not null check(0 < mass_number),     -- A
        state varchar(1) not null,
        zai integer not null check(10010 <= zai) unique,
        half_life real not null check(0 <= half_life),
        primary key(element, mass_number, state)
    );
    
    CREATE TABLE IF NOT EXISTS timestep_nuclide(
        timestep_id integer not null,
        element varchar(2) not null,
        mass_number integer not null,
        state varchar(1) not null,
        atoms real not null,
        grams real not null,
        activity real not null,
        alpha_activity real not null,
        beta_activity real not null,
        gamma_activity real not null,
        heat real not null,
        alpha_heat real not null,
        beta_heat real not null,
        gamma_heat real not null,
        dose real not null,
        ingestion real not null,
        inhalation real not null,
        primary key(timestep_id, element, mass_number, state),
        foreign key(timestep_id) references timestep(id),
        foreign key(element, mass_number, state) references nuclide(element, mass_number, state) 
    );

    CREATE TABLE IF NOT EXISTS timestep_gamma(
        timestep_id integer not null,
        boundary real not null check(0 <= boundary),
        intensity real not null, 
        primary key(timestep_id, boundary),
        foreign key(timestep_id) references timestep(id),
    );
"""
    con.execute(sql)

In [155]:
create_tables(con)

In [85]:
# con.execute("delete from rundata")

In [81]:
def save_rundata(con, ds):
    sql = """
        INSERT INTO rundata values(?, ?, ?, ?, ?)
    """
    con.execute(sql, (    
        ds.coords["timestamp"].item(),
        ds.attrs["run_name"],
        ds.attrs["flux_name"],
        ds.attrs["dose_rate_type"],
        ds.attrs["dose_rate_distance"]
    ))
    con.commit()

In [83]:
save_rundata(con, ds)

In [84]:
con.execute("select * from rundata").df()

,timestamp,run_name,flux_name,dose_rate_type,dose_rate_distance
0,1594594879000000000,"* Material Ag, fluxes 1",55.F9.10 11-L2-02W HFS_GLRY_08_U,Point source,1.0


In [114]:
def save_timesteps(con, ds):
    timesteps_df = ds[
        [
            "time_step_number",
            "elapsed_time",
            "irradiation_time",
            "cooling_time",
            "duration",
            "flux",
            "total_atoms",
            "total_activity",
            "total_alpha_activity",
            "total_beta_activity",
            "total_gamma_activity",
            "total_mass",
            "total_heat",
            "total_alpha_heat",
            "total_beta_heat",
            "total_gamma_heat",
            "total_ingest1ion_dose",
            "total_inhalation_dose",
            "total_dose_rate",
        ]
    ].to_pandas().reset_index()
    sql = "insert into timestep select * from timesteps_df"
    con.execute(sql)
    con.commit()

In [115]:
save_timesteps(con, ds)

In [116]:
con.execute("select * from timestep").df()

,id,elapsed_time,irradiation_time,cooling_time,duration,flux,total_atoms,total_activity,total_alpha_activity,total_beta_activity,total_gamma_activity,total_mass,total_heat,total_alpha_heat,total_beta_heat,total_gamma_heat,total_ingest1ion_dose,total_inhalation_dose,total_dose_rate
0,1,0.0,0.0,0.0,0.0,0.000000e+00,5.577957e+21,0.000000e+00,0.0,0.0,0.0,0.000999,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
1,2,63115200.0,63115200.0,0.0,63115200.0,2.445200e+10,5.577957e+21,6.123276e+09,0.0,0.0,0.0,0.000999,0.000001,0.0,9.894924e-07,1.135242e-07,0.544602,2.323375,0.000092


In [131]:
ds[["element", "mass_number", "state", "zai", "nuclide_half_life"]].to_pandas().reset_index(drop=True)

,element,mass_number,state,zai,nuclide_half_life
0,Ag,105,,471050,3.568320e+06
1,Ag,105,m,471051,4.338000e+02
2,Ag,106,,471060,1.440000e+03
3,Ag,106,m,471061,7.309440e+05
4,Ag,107,,471070,0.000000e+00
5,Ag,107,m,471071,4.430000e+01
6,Ag,108,,471080,1.440000e+02
7,Ag,108,m,471081,1.319110e+10
8,Ag,109,,471090,0.000000e+00
9,Ag,109,m,471091,3.970000e+01


In [156]:
def save_nuclides(con, ds):
    nuclides_df = ds[["element", "mass_number", "state", "zai", "nuclide_half_life"]].to_pandas().reset_index(drop=True)
    sql = "insert into nuclide select * from nuclides_df"
    con.execute(sql)
    con.commit()

In [157]:
save_nuclides(con, ds)

In [160]:
con.execute("select * from nuclide").df().head()

,element,mass_number,state,zai,half_life
0,Ag,105,,471050,3.568320e+06
1,Ag,105,m,471051,4.338000e+02
2,Ag,106,,471060,1.440000e+03
3,Ag,106,m,471061,7.309440e+05
4,Ag,107,,471070,0.000000e+00


In [166]:
ds

<xarray.Dataset>
Dimensions:                 (time_step_number: 2, nuclide: 48, timestamp: 1)
Coordinates:
  * time_step_number        (time_step_number) int64 1 2
  * nuclide                 (nuclide) object MultiIndex
  * element                 (nuclide) object 'Ag' 'Ag' 'Ag' ... 'Ru' 'Ru' 'Ru'
  * mass_number             (nuclide) int64 105 105 106 106 ... 106 102 103 104
  * state                   (nuclide) object '' 'm' '' 'm' '' ... 'm' '' '' ''
    elapsed_time            (time_step_number) float64 0.0 6.312e+07
    zai                     (nuclide) int64 471050 471051 ... 441030 441040
  * timestamp               (timestamp) datetime64[ns] 2020-07-12T23:01:19
Data variables: (12/31)
    irradiation_time        (time_step_number) float64 0.0 6.312e+07
    cooling_time            (time_step_number) float64 0.0 0.0
    duration                (time_step_number) float64 0.0 6.312e+07
    flux                    (time_step_number) float64 0.0 2.445e+10
    total_atoms             (time_step_number) float64 5.578e+21 5.578e+21
    total_activity          (time_step_number) float64 0.0 6.123e+09
    ...                      ...
    nuclide_alpha_heat      (time_step_number, nuclide) float64 nan nan ... 0.0
    nuclide_beta_heat       (time_step_number, nuclide) float64 nan nan ... 0.0
    nuclide_gamma_heat      (time_step_number, nuclide) float64 nan nan ... 0.0
    nuclide_dose            (time_step_number, nuclide) float64 nan nan ... 0.0
    nuclide_ingestion       (time_step_number, nuclide) float64 nan nan ... 0.0
    nuclide_inhalation      (time_step_number, nuclide) float64 nan nan ... 0.0
Attributes:
    run_name:            * Material Ag, fluxes 1
    flux_name:           55.F9.10 11-L2-02W HFS_GLRY_08_U
    dose_rate_type:      Point source
    dose_rate_distance:  1.0

In [203]:
ds[[
"time_step_number",
"element",
"mass_number",
"state",
"nuclide_atoms",
"nuclide_grams",
"nuclide_activity",
"nuclide_alpha_activity",
"nuclide_beta_activity",
"nuclide_gamma_activity",
"nuclide_heat",
"nuclide_alpha_heat",
"nuclide_beta_heat",
"nuclide_gamma_heat",
"nuclide_dose",
"nuclide_ingestion",
"nuclide_inhalation",
]].to_array().drop_indexes(["time_step_number", "nuclide"])
# .reset_coords(names=["time_step_number", "nuclide"])
# .stack(dimensions={"tsn": ["time_step_number", "element", "mass_number", "state"]})

ValueError: cannot remove index(es) 'nuclide', which would corrupt the following index built from coordinates 'nuclide', 'element', 'mass_number', 'state':
PandasIndex(MultiIndex([('Ag', 105,  ''),
            ('Ag', 105, 'm'),
            ('Ag', 106,  ''),
            ('Ag', 106, 'm'),
            ('Ag', 107,  ''),
            ('Ag', 107, 'm'),
            ('Ag', 108,  ''),
            ('Ag', 108, 'm'),
            ('Ag', 109,  ''),
            ('Ag', 109, 'm'),
            ('Ag', 110,  ''),
            ('Ag', 110, 'm'),
            ('Ag', 111,  ''),
            ('Ag', 111, 'm'),
            ('Cd', 106,  ''),
            ('Cd', 107,  ''),
            ('Cd', 108,  ''),
            ('Cd', 109,  ''),
            ('Cd', 110,  ''),
            ('Cd', 111,  ''),
            ('Cd', 111, 'm'),
            ('Cd', 112,  ''),
            ( 'H',   1,  ''),
            ( 'H',   2,  ''),
            ( 'H',   3,  ''),
            ('He',   3,  ''),
            ('He',   4,  ''),
            ('Pd', 104

In [186]:
def save_timestep_nucludes(con, ds):
    timesteps_nuclides_df = ds[[
        "time_step_number",
        "element",
        "mass_number",
        "state",
        "nuclide_atoms",
        "nuclide_grams",
        "nuclide_activity",
        "nuclide_alpha_activity",
        "nuclide_beta_activity",
        "nuclide_gamma_activity",
        "nuclide_heat",
        "nuclide_alpha_heat",
        "nuclide_beta_heat",
        "nuclide_gamma_heat",
        "nuclide_dose",
        "nuclide_ingestion",
        "nuclide_inhalation",
    ]].to_dataframe().reset_index(drop=True)
    sql = "insert into timestep_nuclide select * from timesteps_nuclides_df"
    con.execute(sql)
    con.commit()

In [187]:
save_timestep_nucludes(con, ds)

BinderException: Binder Error: table timestep_nuclide has 17 columns but 18 values were supplied

In [41]:
con.close()